In [ ]:
import pystan
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.spatial import distance
import numpy as np
import arviz as az
from joblib import load, dump

In [ ]:
SF_code = """
data {
    int n;
    int y[n];
    vector[n] logEi;
    vector<lower=0>[n] income;
    vector<lower=0>[n] homevalue;
    vector<lower=0>[n] poverty;
    vector<lower=0>[n] unemployed;
    vector<lower=0>[n] education;
    matrix<lower=0>[n,n] wmat;
}
parameters {
    vector[n] theta;
    vector[n] u;
    real beta1;
    real beta3;
    real beta4;
    real beta5;
    real beta6;
    real<lower=0> sigma2_u;
    real<lower=0> sigma2_v;
}
model {
    sigma2_u ~ inv_gamma(0.0005, 0.5);
    sigma2_v ~ inv_gamma(0.0005, 0.5);
    beta1 ~ uniform(-10, 10);
    beta3 ~ uniform(-10, 10);
    beta4 ~ uniform(-10, 10);
    beta5 ~ uniform(-10, 10);
    beta6 ~ uniform(-10, 10);

    target += -0.5 * n * log(sigma2_u);
    for (i in 1:n) {
        for (j in 1:n) {
            target += -0.5 * (u[i] - u[j])^2 * wmat[i, j] / sigma2_u;
        }
    }

    for (i in 1:n) {
        theta[i] ~ normal(logEi[i] + beta1 * income[i] + beta3 * homevalue[i] + beta4 * poverty[i] + beta5 * unemployed[i] + beta6 * education[i] + u[i], sqrt(sigma2_v));
        y[i] ~ poisson(exp(theta[i]));
    }
}
"""

In [ ]:
# Loading the data

# Shapefiles
shapefile_path = 'data/neighborhoods'
sf_neighborhoods = gpd.read_file(shapefile_path)

# SF case and population data
sf_cases = pd.read_csv('data/master_merged.csv', index_col=False)
sf_cases.rename(columns={'Neighborhood': 'nhood'}, inplace=True)

In [ ]:
sf_cases = sf_cases.drop(41)

In [ ]:
# Calculating distance matrix
sf_neighborhoods_projected = sf_neighborhoods.to_crs(epsg=32610)
sf_neighborhoods_projected['centroid'] = sf_neighborhoods_projected.geometry.centroid

neighborhood_x = sf_neighborhoods_projected['centroid'].geometry.x
neighborhood_y = sf_neighborhoods_projected['centroid'].geometry.y

neighborhood_xy = np.column_stack((neighborhood_x, neighborhood_y))

distance_matrix = distance.cdist(neighborhood_xy, neighborhood_xy, 'euclidean')
distance_matrix /=  1000.0

# Compute weight matrix based off of if they are under 20% quantile
quantile_20 = np.quantile(distance_matrix, 0.2)
weight_matix = (distance_matrix < quantile_20).astype(int)
weight_matix

In [ ]:
quantile_20

In [ ]:
weight_matix[0]

In [ ]:
sf_neighborhoods = pd.merge(sf_neighborhoods, sf_cases, on = 'nhood', how = 'left')

In [ ]:
sf_neighborhoods['Homeless Cases'] = sf_neighborhoods['Homeless Cases'].replace(0,2)
sf_neighborhoods['Homeless Cases']

In [ ]:
data = {
    'n': 41,
    'y': np.array(sf_neighborhoods['Homeless Cases']).astype(int).tolist(),
    'logEi': sf_neighborhoods['Log Expected Cases'].tolist(),
    'income': sf_neighborhoods['Median Household Income'].tolist(),
    #'foreign': sf_neighborhoods['Foreign Born'].tolist(),
    'homevalue': sf_neighborhoods['Median Home Value'].tolist(),
    'poverty': sf_neighborhoods['Percent in Poverty'].tolist(),
    'unemployed': sf_neighborhoods['A_Unemployment Rate'].tolist(),
    'education': sf_neighborhoods['Bachelor\'s degree or higher'].tolist(),
    'wmat': weight_matix.tolist()
}

In [ ]:
sm = pystan.StanModel(model_code=SF_code)

In [ ]:
fit = sm.sampling(data=data, chains=1, iter=15000, warmup=7500, control={'max_treedepth': 14, 'adapt_delta': 0.95})

In [ ]:
samples = fit.extract(permuted=True)  
new_samples = {} 

for key, value in samples.items():
    if len(value.shape) > 1:  
        for i in range(value.shape[1]):
            new_samples[f"{key}_{i}"] = value[:, i]
    else:
        new_samples[key] = value 

samples.update(new_samples)  

keys_to_remove = [key for key, value in samples.items() if isinstance(value, np.ndarray) and len(value.shape) > 1]
for key in keys_to_remove:
    del samples[key]

samples_df = pd.DataFrame(samples)

In [ ]:
plt.figure(figsize=(12, 3))

plt.plot(samples_df['sigma2_u'])
plt.title('Trace of $\Theta_{0}$')  
plt.xlabel('Iteration')
plt.ylabel('$\Theta_{i}$')

In [ ]:
samples_df.to_csv('model_results/stan_model_samples.csv', index=False)

In [ ]:
idata = az.from_pystan(fit)
az.plot_trace(idata, var_names=['theta'])
plt.show()

az.plot_trace(idata, var_names=['beta1'])
plt.show()

az.plot_trace(idata, var_names=['beta3'])
plt.show()

az.plot_trace(idata, var_names=['beta4'])
plt.show()

az.plot_trace(idata, var_names=['beta5'])
plt.show()

az.plot_trace(idata, var_names=['beta6'])
plt.show()

az.plot_trace(idata, var_names=['sigma2_u'])
plt.show()

az.plot_trace(idata, var_names=['sigma2_v'])
plt.show()

Extract the betas for interpretation

In [ ]:
samples_df = pd.read_csv('model_results/stan_model_samples.csv')

In [ ]:
betas = ['beta1', 'beta3', 'beta4', 'beta5', 'beta6']

results = {}
for beta in betas:
    mean = samples_df[beta].mean()
    ci_lower = np.percentile(samples_df[beta], 2.5)
    ci_upper = np.percentile(samples_df[beta], 97.5)
    results[beta] = {'mean': mean, '95% CI': (ci_lower, ci_upper)}

for beta, vals in results.items():
    print(f"{beta}: Mean = {vals['mean']}, 95% Credible Interval = {vals['95% CI']}")